In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/dir_name/KEQA_WSDM19


/content/drive/My Drive/Colab Notebooks/dir_name/KEQA_WSDM19


In [ ]:
!ls

Graph.ipynb  KEQA_WSDM19


In [ ]:
dir_name= 'new_folder'
!mkdir dir_name

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/dir_name/KEQA_WSDM19

/content/drive/My Drive/Colab Notebooks/dir_name/KEQA_WSDM19


In [ ]:
!git clone https://github.com/xhuang31/KEQA_WSDM19.git

Cloning into 'KEQA_WSDM19'...
remote: Enumerating objects: 106, done.
remote: Total 106 (delta 0), reused 0 (delta 0), pack-reused 106
Receiving objects: 100% (106/106), 35.89 KiB | 510.00 KiB/s, done.
Resolving deltas: 100% (61/61), done.


In [ ]:
%cd ..


/


In [ ]:
!ls

augment_process_dataset.py  main.sh	      train_detection.py  util.py
embedding.py		    README.md	      train_entity.py
entity_detection.py	    requirements.txt  train_pred.py
evaluation.py		    test_main.py      trim_names.py


In [ ]:
%%shell
echo "Downloading SimpleQuestions dataset...\n"
wget https://www.dropbox.com/s/9lxudhdfpfkihr1/data.zip
unzip data.zip
rm data.zip

--2020-11-16 11:07:03--  https://www.dropbox.com/s/9lxudhdfpfkihr1/data.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9lxudhdfpfkihr1/data.zip [following]
--2020-11-16 11:07:04--  https://www.dropbox.com/s/raw/9lxudhdfpfkihr1/data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc755c468f3fd030567feba108d8.dl.dropboxusercontent.com/cd/0/inline/BDSb8ICP5kD4qvJm41JFtYCIm62JjQotLIZeAJIeiYqIYmR-xUbh5dBlmVik5M-Icri1pi2XkZEdahm0JHs9CgS09I9tsEBQ9wk99ujnHb0Shu0bZ9ff0ysJ8MUmcjkSAtM/file# [following]
--2020-11-16 11:07:04--  https://uc755c468f3fd030567feba108d8.dl.dropboxusercontent.com/cd/0/inline/BDSb8ICP5kD4qvJm41JFtYCIm62JjQotLIZeAJIeiYqIYmR-xUbh5dBlmVik5M-Icri1pi2XkZEdahm0JHs9CgS09I9tsEBQ9wk99ujnHb0Shu0bZ9ff0ysJ8MU

In [ ]:
%%shell
echo "Preprocess the raw data"
python3.6 trim_names.py -f data/freebase-FB2M.txt -n data/FB5M.name.txt

Preprocess the raw data
get all mids in the Freebase subset...
line: 0
line: 1000000
line: 2000000
line: 3000000
line: 4000000
line: 5000000
line: 6000000
line: 7000000
line: 8000000
line: 9000000
line: 10000000
select head entities based on questions:
line: 0
line: 1000000
line: 2000000
line: 3000000
line: 4000000
line: 5000000
459763 out of 5507279 entities are selected for head
recall of head entity selection: 0.9627001328392645
based on selected entities filter Freebase subset
line: 0
line: 1000000
line: 2000000
line: 3000000
line: 4000000
line: 5000000
line: 6000000
line: 7000000
Number of entities in transE_*: 647676
Number of predicates in transE_*: 4641


In [3]:
!pip install fuzzywuzzy
!pip install torchtext==0.4.0 #other torchtext version have some trouble pickling vocabs

In [6]:
!pip install --pre torch torchtext -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
import nltk
nltk.download('stopwords')

Looking in links: https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
%%shell
echo "Create processed, augmented dataset...\n"
python3.6 augment_process_dataset.py -d data/


Create processed, augmented dataset...\n
/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
Dataset: data/
Index - Names: /preprocess/names.trimmed.txt
Output: preprocess
line: 0
line: 100000
line: 200000
line: 300000
line: 400000
line: 500000
creating new datasets...
reading from data/annotated_fb_data_train.txt
Exact Match Entity : 72558 out of 75910 : 0.9558424450006586
wrote to preprocess/train.txt
reading from data/annotated_fb_data_valid.txt
Exact Match Entity : 10396 out of 10845 : 0.9585984324573537
wrote to preprocess/valid.txt
reading from data/annotated_fb_data_test.txt
Exact Match Entity : 20716 out of 21687 : 0.9552266334670539
wrote to preprocess/test.txt
Total entities 131681
Total predicates 1837
Total words 61396


In [ ]:
%%shell
echo "Embed the Knowledge Graph:\n"
echo "It takes too long time and an existing method is used. Thus, we download the Knowledge Graph Embedding directly...\n"
wget https://www.dropbox.com/s/o5hd8lnr5c0l6hj/KGembed.zip
unzip KGembed.zip
rm KGembed.zip
mv -f KGembed/* preprocess/
rm -r KGembed
#python3.6 transE_emb.py --learning_rate 0.003 --batch_size 3000 --eval_freq 50


Embed the Knowledge Graph:\n
It takes too long time and an existing method is used. Thus, we download the Knowledge Graph Embedding directly...\n
--2020-11-18 03:40:02--  https://www.dropbox.com/s/o5hd8lnr5c0l6hj/KGembed.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/o5hd8lnr5c0l6hj/KGembed.zip [following]
--2020-11-18 03:40:02--  https://www.dropbox.com/s/raw/o5hd8lnr5c0l6hj/KGembed.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc009ce4e9f498d54974a29a9925.dl.dropboxusercontent.com/cd/0/inline/BDacpV768lvpLrq3K_36lWJoqpKH_GCNDrwLZDQEDxeCHI-7fVqckzdNTR6lTTe_QBx_YsQi5zsw9z6dCDW3G2yRUaNZ-mFpJ4wTjr6rX3pD6GzFBJdH1lLeLun5ErRUpag/file# [following]
--2020-11-18 03:40:03--  https://uc009ce4e9f498d54974a29a9925.dl.dropbo

In [ ]:
%%shell
echo "Head Entity Detection (HED) model, train and test the model..."
python3.6 train_detection.py --entity_detection_mode LSTM --fix_embed --gpu 0

Head Entity Detection (HED) model, train and test the model...
Note: You are using GPU for training
Embedding match number 52275 out of 61329
Shift model to GPU
Namespace(batch_size=16, clip_gradient=0.6, cuda=True, dete_prefix='dete', dev_every=12000, entity_detection_mode='LSTM', epochs=30, gpu=0, hidden_size=300, label=4, log_every=2000, lr=0.0003, num_layer=2, output='preprocess/', patience=15, rnn_dropout=0.3, rnn_fc_dropout=0.3, seed=3435, train_embed=False, vector_cache='data/sq_glove300d.pt', weight_decay=0, words_dim=300, words_num=61329)
VOCAB num 61329
Train instance 75709
Dev instance 10845
Entity Type 4
EntityDetection(
  (embed): Embedding(61329, 300)
  (lstm): LSTM(300, 300, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (hidden2tag): Sequential(
    (0): Linear(in_features=600, out_features=600, bias=True)
    (1): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(

In [ ]:
%%shell
echo "Entity representation learning..."
python3.6 train_entity.py --qa_mode GRU --fix_embed --gpu 0
echo "entity_best_model.pt saved at preprocess folder"


Entity representation learning...
getting names map...
Note: You are using GPU for training
Word embedding match number 49580 out of 57922
Shift model to GPU
Namespace(batch_size=32, best_prefix='entity', clip_gradient=0.6, cuda=True, dev_every=10000, embed_dim=250, epochs=30, gpu=0, hidden_size=300, label=250, log_every=2000, lr=0.0002, num_layer=2, output='preprocess', output_channel=300, patience=10, qa_mode='GRU', rnn_dropout=0.3, rnn_fc_dropout=0.3, seed=3435, train_embed=False, vector_cache='data/sq_glove300d.pt', weight_decay=0, words_dim=300, words_num=57922)
VOCAB num 57922
Train instance 75903
Dev instance 4170
EmbedVector(
  (embed): Embedding(57922, 300)
  (gru): GRU(300, 300, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (nonlinear): Tanh()
  (hidden2tag): Sequential(
    (0): Linear(in_features=600, out_features=600, bias=True)
    (1): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): T

In [ ]:
%%shell
echo "predicate representation learning..."
python3.6 train_pred.py --qa_mode GRU --fix_embed --gpu 0

predicate representation learning...
Note: You are using GPU for training
Word embedding match number 52283 out of 62634
Shift model to GPU
Namespace(batch_size=32, best_prefix='pred', clip_gradient=0.6, cuda=True, dev_every=10000, embed_dim=250, epochs=30, gpu=0, hidden_size=300, label=250, log_every=2000, lr=0.0003, num_layer=2, output='preprocess', output_channel=300, patience=12, qa_mode='GRU', rnn_dropout=0.3, rnn_fc_dropout=0.3, seed=3435, train_embed=False, vector_cache='data/sq_glove300d.pt', weight_decay=0, words_dim=300, words_num=62634)
VOCAB num 62634
Train instance 229715
Dev instance 10845
EmbedVector(
  (embed): Embedding(62634, 300)
  (gru): GRU(300, 300, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (nonlinear): Tanh()
  (hidden2tag): Sequential(
    (0): Linear(in_features=600, out_features=600, bias=True)
    (1): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Dro

In [4]:
!pip install torchtext==0.4.0 #other torchtext version have some trouble pickling vocabs

     |████████████████████████████████| 61kB 7.9MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [ ]:
%%shell
echo "We have to run train_detection.py, train_entity.py, train_pred.py first, before running test_main.py..."
python3.6 test_main1.py --gpu 0

We have to run train_detection.py, train_entity.py, train_pred.py first, before running test_main.py...
/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
HIIIIIIIIIIIIIIIIIIIIIIIIIIIII
Note: You are using GPU for testing
total num of example: 21687
Dev Precision:  91.208791% Recall:  94.700912% F1 Score:  92.922054%
line: 0
line: 1000000
line: 2000000
line: 3000000
line: 4000000
line: 5000000
306 out of 21687 nonmatching names, matching accuracy: 0.9858901646147462
tcmalloc: large alloc 1295319040 bytes == 0x11838e000 @  0x7fa17103b1e7 0x7fa1216685e1 0x7fa1216cf420 0x7fa1216cf682 0x7fa1216cfb3e 0x7fa1216d0395 0x7fa12176765d 0x50a4a5 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 

In [12]:
%%shell
echo "We have to run train_detection.py, train_entity.py, train_pred.py first, running test_main.py  before interaction_test1.py..."
python3.6 interaction_test1.py --gpu 0

We have to run train_detection.py, train_entity.py, train_pred.py first, running test_main.py  before interaction_test1.py...
/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
Note: You are using GPU for testing
Please Ask Question:
0


In [5]:
!pip install --pre torch torchtext -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
import nltk
nltk.download('stopwords')

Looking in links: https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import os
import pickle
TEXT=[]
ED=[]
field_t1=open(os.path.join('./preprocess','TEXT.pkl'),'wb')
field_e1=open(os.path.join('./preprocess','ED.pkl'),'wb')
pickle.dump(TEXT,field_t1)
pickle.dump(ED, field_e1)
field_t1.close()
field_e1.close()
##########################

In [ ]:
!pip install torchtext==0.4.0

     |████████████████████████████████| 61kB 6.4MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1
